# Jupiter Notebook to test csv_stuff.py

In [43]:
from operator import ne
import pandas as pd
import sys
import re
import random
from bs4 import BeautifulSoup
import argparse
import matplotlib.pyplot as plt
import difflib

# df_labled = []


def create_labled_table(df, semester, ha, tasks, prog_language):
    df_labled = pd.DataFrame(columns=['semester', 'ha', 'task', 'prog_lang',
                             'surname1', 'lastname1', 'surname2', 'lastname2', 'code1', 'code2', 'label', 'hand_labled'])
    # df_labled.index.name = 'index'
    for task in tasks:
        i = 0
        for ln1, sn1, code1 in df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values:
            for ln2, sn2, code2 in df.loc[(df[f'{task} empty'] == 0) & (df[task] != code1)][['Nachname', 'Vorname', task]][i:].values:
                prelabel1 = difflib.SequenceMatcher(None, code1, code2).ratio()
                prelabel2 = difflib.SequenceMatcher(None, code2, code1).ratio()
                df_labled.loc[len(df_labled)] = [
                    semester, ha, task, prog_language, sn1, ln1, sn2, ln2, code1, code2, max(prelabel1, prelabel2), 0]
            i += 1
    return df_labled


def get_given_code(file):
    try:
        with open(file) as xmlstr:
            soup = BeautifulSoup(xmlstr, 'xml')
            answerpreload = soup.find('answerpreload').text
            questiontext = soup.find('questiontext').text
            return answerpreload, questiontext
    except FileNotFoundError:
        return "Keine Vorgabedatei im Repo gefunden", "Keine Vorgabedatei im Repo gefunden"


"""
(empty- or template solution) == 1
"""


def add_valid_code_columns(df, semester, ha, tasks, prog_language):
    for task in tasks:
        # answerpreload = get_given_code(f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
        answerpreload, _ = get_given_code(
            f'./data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
        answerpreload = answerpreload.replace(
            '\t', '').replace('\r', '').replace('\n', '')
        # every special character beetween \Q and \E is ignored, but the inserted \S*; answers shouldnt contain \Q and \E!
        answerpreload = re.escape(answerpreload)
        # {{ cr_random.f1 }} --> \S*
        answerpreload = re.sub(
            r"\\{\\{\\\s*\S+\s*\\}\\}", r"\\S*", answerpreload)
        summe = 0
        result = 0
        df[f"{task} empty"] = 0
        # TODO wie vergleicht man leicht unterschiedliche texte auf Gleichheit miteinander??? -> bisher nur direkte gleichheit
        for j, column in enumerate(df[task]):
            # there are \\n in the student solutions because "...\n" --> "...\\n" while read_csv
            column = column.replace('\t', '').replace(
                '\r', '').replace('\n', '').replace('\\n', r'\n')
            # test if template was given or not
            if answerpreload != '':
                result = re.match(answerpreload, column)
            if column == '-' or result:
                df.loc[j, f"{task} empty"] = 1
                summe += 1
    return df


def create_labled_table_routine(semester, ha, tasks, prog_language, labled_csv=None):
    # csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    csv_path = f'./data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    df = pd.read_csv(csv_path, delimiter=',')
    keep_columns = ['Nachname', 'Vorname'] + tasks
    drop_columns = []
    for x in df.columns:
        if x not in keep_columns:
            drop_columns.append(x)
    df = df.drop(drop_columns, axis=1)
    df = add_valid_code_columns(
        df=df, semester=semester, ha=ha, tasks=tasks, prog_language=prog_language)
    df_labled = create_labled_table(df, semester, ha, tasks, prog_language)
    return df_labled, len(df_labled)


def get_new_pair(df_labled, last_task, last_id):
    next = df_labled.iloc[last_id:].loc[df_labled['hand_labled'] == 0]
    if next.empty:
        if last_id != 0:
            return get_new_pair(df_labled, last_task, 0)
        return None
    index = next.index[0]
    next = next.values[0]
    answerpreload = None
    questiontext = None
    # return new template
    if last_task != next[2]:
        answerpreload, questiontext = get_given_code(
            f'./data/code_templates/PPR [{next[0]}]-{next[1]}. Hausaufgabe - Pflichttest {next[3]}-Antworten_{next[2]}.xml')
        if answerpreload == "":
            answerpreload = "Keine Vorgabe"
    return next[8], next[9], answerpreload, questiontext, next[10], str(index), next[2]



#TODO 1. erfolgreichen match hinbekommen        *check*
#TODO 2. empty_solution_matrix richtig setzen   *check*
#TODO 3. dann daraus ein paar ableiten          *check*
#TODO 4. dann daraus eine tabellenreihe für ...labled.csv machen *check*
#TODO 5. dann eine ganze tabelle draus machen   *check*
#TODO 6. wo wird tabelle zwischengespeichert?
#TODO 7.1 funktion schreiben die die erste Routine macht mit #aller paar und #aller bereits gelableten und die 3 texte ausgibt
#TODO 7. funktion schreiben, die ein lable übergeben bekommt, in tabelle einträgt und ein neues paar zurückgibt (ggf. auch neue vorgabe)
#TODO gibt es ein Download fenster?
#TODO checken weshalb nur 12/13 leere abgaben bei antwort 10 gefunden wurden
#TODO beim überspringen erstmal mit -1 lablen?; double linked list, für die id schreiben, um prev und next button zu realisieren


In [ ]:
semester='SoSe21'
ha='9'
# tasks = ['Antwort 9']
tasks = ['Antwort 8', 'Antwort 9', 'Antwort 10']
prog_language='C'
df_labled, count = create_labled_table_routine(semester=semester, ha=ha, tasks=tasks, prog_language=prog_language)
# answerpreload = get_given_code(f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{tasks[0]}.xml')

In [46]:
semester='SoSe21'
ha=['6','7','8','9','10']   #5 relevant programm homework weeks + (possible exam tasks)
abgaben=[34,34,34,34,34]    #SoSe2021 34 student only in c
tasks = [4, 3, 3, 3, 3]     #2-4 programm tasks per week
summe=0
for i,j in zip(abgaben,tasks):
    summe += (i-1)**2*j     #with 34 comes 33 pairs per student per task per week
print(summe)                #number of homework pairs in one semester
print(summe*4)              #number of pairs in four semester
print(summe*4*2)            #number of pairs in four semester in 2 programming languages


17424
69696
139392


## Test difflib with hand-labled data

In [2]:
import difflib
import sys
from os import listdir
from os.path import isfile, join
import math

def difflib_on_directory(path):
    file_names = [f for f in listdir(path) if isfile(join(path, f))]
    if len(file_names) < 2:
        print('no files to compare')
    file_contents = [open(f).readlines() for f in file_names]
    num_files = len(file_names)
    sims = []
    for x in range(0, num_files):
        for y in range(x+1, num_files):
            s = difflib.SequenceMatcher(None, file_contents[x], file_contents[y])
            ratio = s.ratio()
            sims.append((file_names[x], file_names[y], ratio))

    sorted_sims = sorted(sims, key=lambda s: -s[2])

    for f1, f2, score in sorted_sims:
        print(f1 + ',' + f2 + ',' + str(score))


In [3]:
# creates c files out of csv's
df = pd.read_csv('~/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe21]-7. Hausaufgabe - Pflichttest C-Antworten_labled.csv', delimiter=',')
sum = 0
for row in df[['lastname1','code1']].values:
    if sum == 15:
        # print((row[0], row[1]))
        sum = 0
        with open(f"{row[0]}.c", "w") as file:
            file.write(row[1])
    sum = sum + 1


In [8]:
difflib_on_directory("/home/erik/TU/ni/plagiate_labeltool/data/labled/c_files/")

Hendel.c,Gillich.c,0.39473684210526316
Horneburg.c,Bauszus.c,0.36923076923076925
Horneburg.c,Durak.c,0.34210526315789475
Hendel.c,Platena.c,0.3333333333333333
Bauszus.c,Otto.c,0.31746031746031744
Hendel.c,Gföhler.c,0.3146067415730337
Hansel.c,Durak.c,0.2988505747126437
Miskovic.c,Gillich.c,0.2891566265060241
Miskovic.c,Platena.c,0.2857142857142857
Gillich.c,Platena.c,0.27906976744186046
Miskovic.c,Hendel.c,0.2716049382716049
Hansel.c,Gföhler.c,0.2708333333333333
Platena.c,Gföhler.c,0.26262626262626265
Alfakir.c,Hendel.c,0.2571428571428571
Durak.c,Gföhler.c,0.25263157894736843
Gillich.c,Gföhler.c,0.24175824175824176
Durak.c,Bauszus.c,0.24
Durak.c,Schulz.c,0.23129251700680273
Hansel.c,Schulz.c,0.22972972972972974
Roßmöller-Felz.c,Jacoby.c,0.22641509433962265
Alfakir.c,Zakusek.c,0.22535211267605634
Hendel.c,Eckardt.c,0.22535211267605634
Alfakir.c,Platena.c,0.225
Hendel.c,Durak.c,0.225
Hansel.c,Hendel.c,0.2222222222222222
Durak.c,Platena.c,0.2222222222222222
Hansel.c,Platena.c,0.2197802197

## Create auto-labeld data with difflib

In [1]:
import pandas as pd
csv_path = "/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe22]-9. Hausaufgabe - Pflichttest C-Antworten_labled.csv"
csv_path2 = "/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [SoSe22]-9. Hausaufgabe - Pflichttest C-Antworten_autolabled.csv"
df = pd.read_csv(csv_path, delimiter=',')
# df.to_csv(csv_path)


In [ ]:
import pandas as pd
import difflib

def label_csv_with_difflib(csv_path):
    try:
        df = pd.read_csv(csv_path, delimiter=',')
        for i, row in enumerate(df[['code1', 'code2']].values):
            s = difflib.SequenceMatcher(None, row[0], row[1])
            df.loc[i, ['label']] = s.ratio()
           
    except:
        return f"Couldnt open {csv_path}"


In [ ]:
for row in df.nlargest(20, columns=['label'])[['Unnamed: 0', 'surname1', 'surname2', 'label', 'difflib_']].values:
    print(f"{row[1]}, {row[2]}, {row[3]}, {row[4]}")


In [ ]:
import numpy as np

for row in df.nlargest(20, columns=['label'])[['Unnamed: 0', 'surname1', 'surname2', 'label']].values:
    print(f"{row[1]}, {row[2]}, {row[3]}, {np.abs(df.loc[(df['surname1']==row[2]) & (df['surname2']==row[1])]['label'].values[0] - row[3])}")


## Create auto-labeld data with Levenshtein

In [12]:
from Levenshtein import distance
from sklearn.manifold import MDS

def distance_percentage(string1, string2):
    return distance(string1, string2) / np.max([len(string1), len(string2)])


def get_similarity(string1, string2):


    distance1 = distance_percentage(string1, string2)

    string1_p = string1
    string1 = string1.split()
    string1_w = "".join(string1)
    string1_s = "".join(sorted(string1_w))

    string2_p = string2
    string2 = string2.split()
    string2_w = "".join(string2)
    string2_s = "".join(sorted(string2_w))

    distance2 = distance_percentage(string1_w, string2_w)
    distance3 = distance_percentage(string1_s, string2_s)


    return 1 - distance2#, 1 - distance3


In [13]:
for i, row in enumerate(df[['code1', 'code2']].values):
    label = get_similarity(row[0], row[1])
    # if label >= 0.6:
    #     label = np.round(label)
    # elif label <= 0.35:
    #     label = np.floor(label)
    df.loc[i, ['Levenshtein2']] = label
# df.to_csv(csv_path2)


In [ ]:
df_labled = pd.DataFrame(columns=['semester', 'ha', 'task', 'prog_lang',
                            'surname1', 'lastname1', 'surname2', 'lastname2', 'code1', 'code2', 'label'])
df_labled.index.name = 'index'
for task in tasks:
    for i, ln1, sn1, code1 in enumerate(df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values):
        for ln2, sn2, code2 in df.loc[(df[f'{task} empty'] == 0) & (df[task] != code1) & ()][['Nachname', 'Vorname', task]][i:].values:
            df_labled.loc[len(df_labled)] = [
                semester, ha, task, prog_language, sn1, ln1, sn2, ln2, code1, code2, -1]


In [19]:
for i,j in enumerate(range(0,3)):
    print((i,j))
df[0:2]

(0, 0)
(1, 1)
(2, 2)


,Unnamed: 0,semester,ha,task,prog_lang,surname1,lastname1,surname2,lastname2,code1,code2,label,Levenshtein,Levenshtein2
0,0,SoSe22,9,Antwort 9,C,Fabian,Große-Wöhrmann,Moritz Sebastian,Niethammer,#include <stdio.h>\r\n#include <stdlib.h>\r\n ...,#include <stdio.h>\r\n#include <stdlib.h>\r\n\...,0.521108,0.673325,0.761993
1,1,SoSe22,9,Antwort 9,C,Fabian,Große-Wöhrmann,Florian,Schacht,#include <stdio.h>\r\n#include <stdlib.h>\r\n ...,//gcc -x c HA09_F09.c -o HA09_F09 -Wall -Wextr...,0.051832,0.157407,0.156506


## Postprocess csv file

### Delete rows consist of a pair with answerpreload from another task

In [39]:
import pandas as pd
csv_path = f'../../data/labled/PPR [WS2021]-9. Hausaufgabe - Pflichttest C-Antworten_labled.csv'
df = pd.read_csv(csv_path, delimiter=',')
df = df.drop('Unnamed: 0', axis=1)
df.head(4)


,semester,ha,task,prog_lang,surname1,lastname1,surname2,lastname2,code1,code2,label,hand_labled
0,WS2021,9,Antwort 10,C,Jolanda Marietta,Schumann,David,Stubler,#include <stdio.h>\r\n\r\nvoid aussortieren (c...,#include <stdio.h>\r\n#include <stdlib.h>\r\n\...,0.0,1
1,WS2021,9,Antwort 10,C,Jolanda Marietta,Schumann,Daniel Dominik,Eichstädt,#include <stdio.h>\r\n\r\nvoid aussortieren (c...,#include <stdio.h>\r\n\r\nchar aussortieren (c...,0.0,1
2,WS2021,9,Antwort 10,C,Jolanda Marietta,Schumann,Mohamad Anas,Allaham,#include <stdio.h>\r\n\r\nvoid aussortieren (c...,#include <stdio.h>\r\n#include <stdlib.h>\r\n\...,0.0,1
3,WS2021,9,Antwort 10,C,Jolanda Marietta,Schumann,Moritz,Wahl,#include <stdio.h>\r\n\r\nvoid aussortieren (c...,#include <stdio.h>\r\n#include <stdlib.h>\r\n ...,0.0,1


In [ ]:
semester='WS2021'
ha='9'
prog_language='C'
task='Antwort 9'
answerpreload, _ = get_given_code(
            f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
print(answerpreload)

In [34]:

def compare_with_given_code(student_code, semester, ha, task, prog_language):
    # answerpreload = get_given_code(f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
    answerpreload, _ = get_given_code(
        f'../../data/code_templates/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_{task}.xml')
    answerpreload = answerpreload.replace(
        '\t', '').replace('\r', '').replace('\n', '')
    # every special character beetween \Q and \E is ignored, but the inserted \S*; answers shouldnt contain \Q and \E!
    answerpreload = re.escape(answerpreload)
    # {{ cr_random.f1 }} --> \S*
    answerpreload = re.sub(
        r"\\{\\{\\\s*\S+\s*\\}\\}", r"\\S*", answerpreload)
    # there are \\n in the student solutions because "...\n" --> "...\\n" while read_csv
    student_code = student_code.replace('\t', '').replace(
        '\r', '').replace('\n', '').replace('\\n', r'\n')
    # test if template was given or not
    if answerpreload != '':
        result = re.match(answerpreload, student_code)
    return result != None


In [62]:
# summe=0
# for i, (code1, code2) in enumerate(df[['code1','code2']].values):
#     compare1 = compare_with_given_code(code1,ha=ha,prog_language=prog_language,semester=semester,task=task)
#     compare2 = compare_with_given_code(code2,ha=ha,prog_language=prog_language,semester=semester,task=task)
#     # print((i, lastname1, lastname2, compare1, compare2))
#     if compare1 or compare2:
#         df = df.drop(i)
#         summe+=1
# print(summe)
df.to_csv(f'/home/erik/TU/ni/plagiate_labeltool/data/labled/PPR [WS2021]-9. Hausaufgabe - Pflichttest C-Antworten_labled_2.csv')
